In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Fri Aug  6 14:08:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import os
import requests

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
# download gpt2 models.  
gpt2.download_gpt2(model_name="124M")
gpt2.download_gpt2(model_name="355M")
#gpt2.download_gpt2(model_name="774M")
#gpt2.download_gpt2(model_name="1558M")

Fetching checkpoint: 1.05Mit [00:00, 237Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:01, 814kit/s]
Fetching hparams.json: 1.05Mit [00:00, 343Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [01:02, 8.02Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 189Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 1.14Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 1.14Mit/s]
Fetching checkpoint: 1.05Mit [00:00, 217Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:01, 946kit/s]
Fetching hparams.json: 1.05Mit [00:00, 287Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [05:52, 4.02Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 187Mit/s]                                                
Fetching model.ckpt.meta: 1

In [ ]:
#  copy models to google drive if you want
!cp -r /content/models /content/drive/MyDrive 

In [5]:
# load dataset
file_name = "HarryPotter.txt"
if not os.path.isfile(file_name):
	url = "https://raw.githubusercontent.com/YanWang20121865/MSc2021YanWang/main/Second%20update/Dataset/HarryPotter.txt"
	data = requests.get(url)
	
	with open(file_name, 'w') as f:
		f.write(data.text)

In [6]:
!export PYTHONIOENCODING=UTF-8

In [7]:
# load checkpoint to google drive if in need( if you want to restart training from existed checkpoint)
!cp -r /content/drive/MyDrive/checkpoint/ /content/checkpoint/

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1500,
              #Set to fresh to start training from the base GPT-2, or set to latest to restart training from an existing checkpoint.
              restore_from='latest',   
              run_name='run_124_HarryPotter',
              print_every=10,
              sample_every=200,
              save_every=500,
              
              )

In [9]:
# save checkpoint to google drive
!cp -r /content/checkpoint/ /content/drive/MyDrive

In [8]:
# load the model we trained
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run_124_HarryPotter')

Loading checkpoint checkpoint/run_124_HarryPotter/model-1627
INFO:tensorflow:Restoring parameters from checkpoint/run_124_HarryPotter/model-1627


In [11]:
# generate txt files to save stories
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      run_name='run_124_HarryPotter',
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )